# Create a meshed screen with a Gaussian beam

The screen is placed at the origin in the x-y-plane.<br>
The screen normal is pointing in the negative z direction (x - right, y - up).<br>
The beam is propagating in positive z direction.

if not yet available some libraries have to be installed :<br>
- gmsh (best installed globally through package management system)
- python3 -m pip install pygmsh --user
- python3 -m pip install vtk --user

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import vtk

import pygmsh
import numpy as np
from scipy import constants
import h5py

In [1]:
import numpy as np
from scipy import constants
import MeshFields as MF

In [2]:
pts, tris = MF.CircularMesh(R=0.04, ratio=1.0, lcar=0.02)
print("%d points" % len(pts))
print("%d triangles" % len(tris))
pos = [(pts[t[0]] + pts[t[1]] + pts[t[2]]) / 3.0 for t in tris]
NP = len(pos)
print("%d source positions" % NP)

10660 points
20998 triangles
20998 source positions


In [3]:
area = MF.MeshArea(pts, tris)
print(np.min(area))
print(np.max(area))
print(np.sum(area))

1.06957750384e-07
5.40974664727e-07
0.00502621704173


In [4]:
normals = MF.MeshNormals(pts, tris)
print(np.sum(normals, axis=0))

[     0.      0.  20998.]


### define the beam parameters

In [5]:
# time step
dt=2.0e-14
NOTS=400

# wave parameters
tau = 4.0e-12
f = 0.3e12
print("f = %.2f THz" % (f*1e-12))
w0 = 0.02
lam = constants.c/f
print("λ = %.2f µm" % (lam*1e6))
zR = np.pi*w0*w0/lam
print("w0 = %.2f mm" % (1e3*w0))
print("Rayleigh range = %.3f m" % zR)


f = 0.30 THz
λ = 999.31 µm
w0 = 20.00 mm
Rayleigh range = 1.258 m


### compute the field traces for every point

In [6]:
# all points use the same timing grid
t = np.linspace(-NOTS/2*dt, (NOTS/2-1)*dt, NOTS)
t0 = [-NOTS/2*dt for dummy in t]
# field (6 components) traces on screen
A = np.zeros((NP,NOTS,6))
# electric field amplitude in V/m
E0 = 2.0e7
# magnetic field amplitude in T
B0 = E0 / constants.c

for ip, p in enumerate(pos):
    r = np.sqrt(p[0]*p[0]+p[1]*p[1])
    # field strength
    osc = np.cos(2.0*np.pi*f*t) * np.exp(-r*r/(w0*w0)) * np.exp(-np.square(t)/(2.0*tau*tau))
    Ex = E0 * osc
    Ey = np.zeros_like(Ex)
    Ez = np.zeros_like(Ex)
    Bx = np.zeros_like(Ex)
    By = B0 * osc
    Bz = np.zeros_like(Ex)
    trace = np.array([Ex,Ey,Ez,Bx,By,Bz]).transpose()
    A[ip] = trace

In [7]:
# compute power density for plotting
Pz = []
for trace in A:
    EVec = trace[:,0:3]
    BVec = trace[:,3:6]
    SVec = np.cross(EVec, BVec) / constants.mu_0
    Pz.append((SVec.sum(axis=0))[2]*dt)

print("peak energy density = %.6f J/m²" % np.max(Pz))
print("total pulse energy = %.1f µJ" % (1e6*np.dot(area,Pz)))

peak energy density = 3.242731 J/m²
total pulse energy = 2037.2 µJ


### display mesh

In [9]:
MF.ShowMesh(pts,tris,pos)

In [10]:
MF.ShowMesh(pts,tris,scalars=Pz,scalarTitle="Pz")

In [12]:
filename="Mesh.h5"
MF.WriteMesh(filename, pts,tris,pos)

In [13]:
filename="MeshGaussian.h5"
MF.WriteMeshField(filename, pts,tris,pos,t0,dt,A)

In [15]:
filename="MeshGaussian.h5"
pts, tris, pos, t0, dt, A = MF.ReadMeshField(filename)

# compute power density for plotting
Pz = []
for trace in A:
    EVec = trace[:,0:3]
    BVec = trace[:,3:6]
    SVec = np.cross(EVec, BVec) / constants.mu_0
    Pz.append((SVec.sum(axis=0))[2]*dt)

print("peak energy density = %.6f J/m²" % np.max(Pz))
print("total pulse energy = %.1f µJ" % (1e6*np.dot(area,Pz)))

MF.ShowMesh(pts,tris,scalars=Pz,scalarTitle="Pz")

peak energy density = 3.242731 J/m²
total pulse energy = 2037.2 µJ
